In [25]:
import os


In [9]:
!curl -L -o ~/code/ViniciusGalina1995/skin_lens/raw_data/multiple-skin-disease-detection-and-classification.zip\
  https://www.kaggle.com/api/v1/datasets/download/pritpal2873/multiple-skin-disease-detection-and-classification

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  784M  100  784M    0     0  25.6M      0  0:00:30  0:00:30 --:--:-- 24.7M 0:00:30  0:00:24  0:00:06 26.4M


In [13]:
!unzip -qn ~/code/ViniciusGalina1995/skin_lens/raw_data/multiple-skin-disease-detection-and-classification.zip -d ~/code/ViniciusGalina1995/skin_lens/raw_data/


In [14]:
import splitfolders

input_folder = r'/home/vinicius/code/ViniciusGalina1995/skin_lens/raw_data/Skin Cancer Dataset'

# Split with a ratio
# To only split into training and validation set, set a tuple to ratio, i.e; 80/20
#Train, val, test
splitfolders.ratio(input_folder, output=r'/home/vinicius/code/ViniciusGalina1995/skin_lens/raw_data/Split', 
                   seed=42, ratio=(.7, .1, .2), 
                   group_prefix=None) # default values

Copying files: 4109 files [00:03, 1175.81 files/s]


In [31]:
train_dir = os.path.join("..", "raw_data", "Split", "train")
val_dir = os.path.join("..", "raw_data", "Split", "val")
test_dir = os.path.join("..", "raw_data", "Split", "test")
batch_size = 64

In [22]:
from tensorflow.keras.utils import image_dataset_from_directory # type: ignore

def load_data(data_dir: str):

    ds = image_dataset_from_directory(
    data_dir,
    labels="inferred",
    label_mode="categorical",
    seed=123,
    image_size=(150, 150),
    batch_size=batch_size)

    return ds

In [27]:
train_ds = load_data(train_dir)


Found 2874 files belonging to 9 classes.


2025-01-28 12:03:20.266955: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-01-28 12:03:20.267343: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-01-28 12:03:20.267501: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2025-01-28 12:03:20.267945: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2025-01-28 12:03:20.268788: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

In [28]:
train_ds.class_names

['Acitinic Keratosis',
 'Basal Cell Carcinoma',
 'Dermatofibroma',
 'Melanoma',
 'Nevus',
 'Pigmented Benign Keratosis',
 'Seborrheic Keratosis',
 'Squamous Cell Carcinoma',
 'Vascular Lesion']

In [29]:
type(train_ds)

tensorflow.python.data.ops.dataset_ops.BatchDataset